In [ ]:
import retinanalysis as ra
import retinanalysis.config.schema as schema
import retinanalysis.classes.dedup as dd
import importlib
importlib.reload(dd)
import numpy as np
import matplotlib.pyplot as plt

### First, let's pull out a dataframe showing noise runs from a specific experiment day. 
### We can choose this day by querying for specific protocols with specific parameters. 

In [ ]:
df = ra.get_datasets_from_protocol_names('.SpatialNoise')
df_1 = df[df['exp_name'] == '20250429C']
display(df_1)

### Let's look at the first datafile from the first chunk.

In [ ]:
sb = ra.StimBlock(df_1.at[172, 'exp_name'], df_1.at[172, 'block_id'])
ac = ra.AnalysisChunk(df_1.at[172, 'exp_name'], df_1.at[172, 'chunk_name'])

#### What ei correlation method should we use? We can compare the results before instantiating our deduplication object.

In [ ]:
axs, ei_full_pairs, ei_space_pairs, ei_power_pairs = dd.compare_ei_methods(ac, ei_threshold=0.9)

#### Space seems like a middle ground. Let's use that for now.

In [ ]:
db = dd.DedupBlock(exp_name=df_1.at[172, 'exp_name'], chunk_name=df_1.at[172, 'chunk_name'], ss_version='kilosort2.5', \
                   pkl_file=None, is_noise=True, ei_method='space', ei_threshold=0.9, sm_threshold=0.9)
db

### Now we can plot the autocorrelation matrix and histograms. Cluster pairs with correlations above threshold are potentially merger targets.

In [ ]:
db.plot_correlations(axs=None);

#### We can also plot our cluster RFs, and highlight those with above-threshold correlations with another cluster.

In [ ]:
db.plotRFs_dedup(axs=None);

#### We have all sorts of groupings of cells that are highly correlated by ei, spatial map, or both below the hood. We can generate some tables of pairwise statistics for these various groups.

In [ ]:
print(db.dedup_sets.keys())

In [ ]:
summary_stats_all = db.get_summary_stats(pairs=db.dedup_sets['all_pairs'])
summary_stats_all

In [ ]:
summary_stats_both = db.get_summary_stats(pairs=db.dedup_sets['both_pairs'])
summary_stats_both

In [ ]:
summary_stats_ei_space = db.get_summary_stats(pairs=db.dedup_sets['ei_only_pairs'])
summary_stats_ei_space

In [ ]:
summary_stats_sm = db.get_summary_stats(pairs=db.dedup_sets['sm_only_pairs'])
summary_stats_sm

####  since overlap is low between high ei and high sm pairs, let's visualize any transitively connected groups across either criterion

In [ ]:
for pair in db.dedup_sets['extended_pairs_across']:
    db.visualize_groups(pair, axs=None);
    db.plot_amplitude_histograms(pair, axs=None);

#### 199 and 212 look awfully like the same cell. What do their top PC projections look like (from KS)?

In [ ]:
db.plot_pcs(pair=(199, 212));

#### not always super clear though with a 2D plot. These also have some overlap, and definitely are not the same cell.

In [ ]:
db.plot_pcs(pair=(725, 728));